# VGG + BRAIN TUMOR

In [ ]:
# ! pip install torch torchvision numpy pydantic umap-learn interpret faiss-cpu torchsummary matplotlib pandas seaborn scikit-learn scikit-image pytorch-lightning[extra] --no-cache-dir --ignore-installed

In [1]:
import copy
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torch.optim as optim

from data.brain_tumor_mri import BrainTumorMRI
from models.vgg import VGG
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from torchvision import transforms
from torchvision.transforms import functional as F

In [2]:
RND_SEED=31
torch.manual_seed(RND_SEED)
torch.cuda.manual_seed_all(RND_SEED)
np.random.seed(RND_SEED)

CUDA_AVAILABLE = torch.cuda.is_available()
DEVICE = torch.device("cpu:0")
BATCH_SIZE = 64
NUM_WORKERS = 4
EPOCHS = 30
DATASETS_PATH = './datasets/'
DATASET_RATIO = None

In [3]:
brain_tumor = BrainTumorMRI(DATASETS_PATH, train_size=0.7, val_size=0.3, random_state=555, transforms=transforms.Compose(
            [
                transforms.Resize((255, 255)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(10),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        ))

# MODEL

In [5]:
model = VGG(num_classes=4, freeze=False)

logger = CSVLogger(save_dir="./", name="experiments/vgg", flush_logs_every_n_steps=1)
checkpoint_callback = ModelCheckpoint(
    filename="vgg-{epoch}-{train_loss:.4f}-{train_acc:.4f}-{val_loss:.4f}-{val_acc:.4f}",
    save_top_k=3, save_last=True, monitor="val_loss", mode="min", verbose=True
)
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=5, mode="min", verbose=True)

trainer = Trainer(
    max_epochs=EPOCHS,
    callbacks=[early_stopping_callback, checkpoint_callback],
    log_every_n_steps=1,
    logger=logger,
    limit_train_batches=DATASET_RATIO or 1.0,
    limit_val_batches=DATASET_RATIO or 1.0,
    limit_test_batches=DATASET_RATIO or 1.0
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..


## TRAINING

In [6]:
trainer.fit(
    model,
    train_dataloaders=brain_tumor.train_dataloader,
    val_dataloaders=brain_tumor.val_dataloader
)


  | Name              | Type               | Params
---------------------------------------------------------
0 | feature_extractor | Sequential         | 14.7 M
1 | avgpool           | AdaptiveAvgPool2d  | 0     
2 | classifier        | Sequential         | 119 M 
3 | loss_fn           | CrossEntropyLoss   | 0     
4 | train_acc         | MulticlassAccuracy | 0     
5 | val_acc           | MulticlassAccuracy | 0     
6 | test_acc          | MulticlassAccuracy | 0     
---------------------------------------------------------
134 M     Trainable params
0         Non-trainable params
134 M     Total params
537.108   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [7]:
print(trainer.logger.log_dir)

./experiments/vgg/version_0


## TESTING

In [8]:
batch_preds = trainer.predict(model, dataloaders=brain_tumor.test_dataloader)

Predicting: 0it [00:00, ?it/s]

In [9]:
truth_labels = []
preds_labels = []
for truth, pred in batch_preds:
    truth_labels.append(truth)
    preds_labels.append(pred)
truth_labels = np.concatenate(truth_labels)
preds_labels = np.concatenate(preds_labels)

In [10]:
with open(f"{trainer.logger.log_dir}/vgg-preds.dump", "wb") as f:
    pickle.dump([truth_labels, preds_labels], f)